## Amazon User-profile Puchasing-history-record Crawling
Step1 Crawling of AMZ profile project.

Assign a product and crawl the user's ,who leave their review below, history purchasing record on their profile page on Amazon.
### Before running the crawling code you should:
- Download the browser driver (Chrome) from [selenium web page]('https://chromedriver.chromium.org/downloads') and put it in project folder.
- Make sure you have already pip install the selenium, tqdm, pandas packages.
- Make sure the `internet stable`.
### When running the crawling code you should:
- Follow the step of the instuction in each cell.
- See the selenium browser automatically crawl the data and `do not close the windows of the browser`.
- If any error happend just restart from that page with `STEP 2`.
### After running the crawling code you should:
- Check the if data amount make sence with the reviewers amount.
- Make sure the `File name compare to the model name`.
- Restart the all cell when crawling the new model.

### Table of Content
- ['Packages'](#packages) : Import packages.

Define Function
- ['Open Browser'](#open-browser) : Definie the seting of browser and Open browser.
- ['Get info'](#getinfo) : Scrape the data elements on single page.
- ['Initialize Format'](#initialize-format): Build empty format for saving data.
- ['Crawl Model Review User'](#crawlmodelreviewuser) : Crawl the review page data.
- ['Scroll Down Page'](#scrolldownpage) : For loading dynamic data in user's profile page.
- ['Get user info'](#getuserinfo) : Crawl user purchasing history record data.

Execution
- ['Execution'](#execution) : Execute the Code and Output Data to csv file.



### Packages

In [ ]:
from selenium import webdriver
import pandas as pd
import time


### Open Browser
Open Selenium Browser and Setting

In [ ]:
'''
Arg:
    ModelAsin : Assignment model Asin on Amazon (str)
    
Output:
    Open Browser on All review page of Model on Amazon
'''


def open_browser(ModelAsin):

    # Setting of Brwoser
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    opt = webdriver.ChromeOptions()
    opt.add_argument('--user-agent=%s' % user_agent)
    # Cancel Picture Loading
    # opt.add_argument('blink-settings=imagesEnabled=False')
    # opt.add_argument('--disable-dev-shm-usage')
    # opt.add_argument('--disable-gpu')
    # opt.add_argument('--disable-extensions')
    opt.add_argument('lang=en')

    # Opening Browser with Setting and Assign URL
    global driver
    driver = webdriver.Chrome('./chromedriver', chrome_options=opt)
    driver.implicitly_wait(10)
    res = driver.get(
        f"https://www.amazon.com/product-reviews/{ModelAsin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&language=en_US")
    return res


### Get_info
Scrape the data elements on single page.

In [ ]:
'''
Arg:
    

Output:
    tmp_format : Data on single webpage in Dataframe type (pd.DataFrame)
'''


def get_info():
    # Scrape the data elements on Webpage
    user_name = driver.find_elements_by_class_name('a-profile-name')
    product_name = driver.find_elements_by_xpath(
        '//*[@id="profile-at-card-container"]/div/div[2]/div[2]/div')
    profile_discriptor = driver.find_elements_by_class_name(
        'a-profile-descriptor')
    ratings = driver.find_elements_by_xpath(
        '//*[@id="profile-at-card-container"]/div/div[2]/div[2]/a/div[1]/div/i/span')
    comment = driver.find_elements_by_class_name(
        'a-spacing-small.a-spacing-top-mini.a-color-base.profile-at-review-text.profile-at-review-text-desktop')
    asin = driver.find_elements_by_class_name(
        'a-link-normal.profile-at-product-box-link.a-text-normal')
    tmp_format = pd.DataFrame({'user_name': [], 'product_name': [], 'review_date_place': [
    ], 'review_rating': [], 'review_content': [], 'Asin': []})

    # Avoiding Product no longer available to raise error and still Scrape other data
    k = 0
    for i in range(len(comment)):
        if product_name[i].text != 'Product no longer available':
            try:
                _asin = str(asin[k].get_attribute(
                    'href').split('/dp/')[1].split('?ref')[0])
            except:
                _asin = str(asin[k].get_attribute('href').split(
                    'trackAsin=')[1].split('&ref')[0])
            tmp_format = tmp_format.append({'user_name': user_name[i].text, 'product_name': product_name[i].text, 'review_date_place': profile_discriptor[i].text, 'review_rating': ratings[i].get_attribute('innerHTML'), 'review_content': comment[i].text,
                                            'Asin': _asin}, ignore_index=True)
            k += 1
        else:
            tmp_format = tmp_format.append({'user_name': user_name[i].text, 'product_name': product_name[i].text, 'review_date_place': profile_discriptor[i].text, 'review_rating': ratings[i].get_attribute('innerHTML'), 'review_content': comment[i].text,
                                            'Asin': "None"}, ignore_index=True)

    return tmp_format


### Initialize Format
#### Make Sure Changing the `Required Model Name`

In [ ]:
required_product = ['SAMSUNG 49-inch Odyssey G9 Gaming Monitor']
total_format = pd.DataFrame({'user_name': [], 'product_name': [], 'review_date_place': [
], 'review_rating': [], 'review_content': [], 'Asin': []})



### Crawl_model_review_user
Crawl the review page data.

In [ ]:
'''
Arg:
    

Output:
    review_format : Data on review webpages in Dataframe type (pd.DataFrame)
'''

def Crawl_model_review_user():  # 抓取model的評論資料
    review_format = pd.DataFrame({'user_name': [], 'product_name': [], 'review_date_place': [
], 'review_rating': [], 'review_content': [], 'Asin': []})
    for p in range(50):  # numbers of review pages
        reviewer = driver.find_elements_by_class_name('a-profile-name')
        review_content = driver.find_elements_by_class_name(
            'a-size-base.review-text.review-text-content')
        review_date = driver.find_elements_by_class_name(
            'a-size-base.a-color-secondary.review-date')
        review_star = driver.find_elements_by_css_selector(
            "i[data-hook='review-star-rating']")
        for n in range(2, len(review_content)):
            review_format = review_format.append({'user_name': reviewer[n].text, 'product_name': required_product[0], 'review_date_place': review_date[n].text, 'review_rating': review_star[n].get_attribute(
                'innerHTML'), 'review_content': review_content[n].text}, ignore_index=True)  # add data in format

        return review_format


### Scroll_down_page
For loading dynamic data in user's profile page.

In [ ]:
def __scroll_down_page():
    current_scroll_position, new_height = 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += 15
        driver.execute_script(
            "window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")


### Get_user_info
Crawl user purchasing history record data.

In [ ]:
'''
Arg:
    

Output:
    user_format : Data on user profile webpages in Dataframe type (pd.DataFrame)
'''
def get_user_info(): 
    
    user_format = pd.DataFrame({'user_name': [], 'product_name': [], 'review_date_place': [
    ], 'review_rating': [], 'review_content': [], 'Asin': []})  # build empty format
    user_name_list = driver.find_elements_by_class_name('a-profile-name')
    for i in range(2, len(user_name_list)):
        user_name_list = driver.find_elements_by_class_name('a-profile-name')
        driver.execute_script("arguments[0].click();", user_name_list[i])
        #user_name_list[i].click() # Click the user transit to profile page
        __scroll_down_page() # Scroll down the page to load dynamic data until it ends
        try:
            user_format = user_format.append(get_info())
            driver.back()
        except:
            driver.back()
        continue
        

    return user_format


### Execution

In [ ]:
# STEP1 Input Model Asin and Open Browser
open_browser('B088HH6LW5')  

In [ ]:
# STEP2 Execute Collecting Data
for i in range(50):  # Change the number by review amount / 12
    total_format.append(Crawl_model_review_user())
    total_format = total_format.append(get_user_info())
    print('Total_format checking...', len(total_format), end='\r')
    driver.find_element_by_xpath(
        '//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a'
    ).click()  # next review page
    time.sleep(3)


In [ ]:
#STEP3 Checking Data Amount and Content
print(total_format.describe) 


In [ ]:
#STEP4 Output the csv File with Model name
total_format.to_csv('AMZ_PH_STEP1' +
                    required_product[0]+'.csv', encoding="utf_8_sig")
